In [48]:
import openai
import json
import dotenv
import os
import pprint

dotenv.load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

pp = pprint.PrettyPrinter(indent=4)

Define a couple of functions to call later

In [75]:
def collatz_count(n: int) -> int:
    count = 0
    while n > 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3*n + 1
        count += 1
    return count

def fibonacci(n: int) -> int:
    if n == 0 or n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    

Here we send the usual, a system message (which seems to not be required, the demo from openai didn't include one), a user question, but **also** a list of functions that can be called and a schema for them.

In [71]:
question = "What's the 7th Fibonacci number?"

response = openai.ChatCompletion.create(
    model="gpt-4-0613", temperature = 0, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}],
    functions=[
        {
            "name": "collatz_count",
            "description": "Counts how many steps it takes for a number to reach 1 in the Collatz sequence.",
            "parameters": {
                "type": "object",
                "properties": {
                    "n": {
                        "type": "integer",
                        "description": "The index of the item in the Collatz sequence to get.",
                    }
                },
                "required": ["n"],
            },
        },
        {
            "name": "fibonacci",
            "description": "Gets the nth number in the Fibonacci sequence.",
            "parameters": {
                "type": "object",
                "properties": {
                    "n": {
                        "type": "integer",
                        "description": "The index of the item in the Fibonacci sequence to get.",
                    }
                },
                "required": ["n"],
            },
        }
    ],
    function_call="auto",
)

pp.pprint(response)


<OpenAIObject chat.completion id=chatcmpl-7RNqc24ZwCCY6J01OszvHJ0zrtvqs at 0x1148f5d90> JSON: {
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"n\": 7\n}",
          "name": "fibonacci"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686760882,
  "id": "chatcmpl-7RNqc24ZwCCY6J01OszvHJ0zrtvqs",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 125,
    "total_tokens": 141
  }
}


*Note* that this returned `null` in the `"content"` which usually holds the model response, instead there's a `"function_call"` response with a JSON string for the function name and args to call. (I've seen the model return info in the content as well though when asking tricky questions.)

So let's extract the info and make the call. Notice that we now include a new message with a new role type of `"function"` and a `"name"` field.

In [74]:
message = response["choices"][0]["message"]

if message.get("function_call"):
    # extract the function name the model wants to call
    function_name = message["function_call"]["name"]
    # extract the params JSON (NB the response from the model may not be valid JSON)
    arguments = json.loads(message["function_call"]["arguments"])

    # lookup the function in the global namespace from the name
    if function_name in globals():
        function = globals()[function_name]
    else:
        raise ValueError(f"Function {function_name} not found.")

    # call the function with the arguments
    function_response = function(**arguments)

    # return the function result to the model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": f"{function_response}",
        },
    ]

    print("Messages:")
    pp.pprint(messages)

    # Step 4, send model the info on the function call and function response
    second_response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages
    )

    print("\n\nResponse:")
    pp.pprint(second_response)

Messages:
[   {'content': 'You are a helpful assistant.', 'role': 'system'},
    {'content': "What's the 7th Fibonacci number?", 'role': 'user'},
    <OpenAIObject at 0x1148f5c10> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"n\": 7\n}",
    "name": "fibonacci"
  },
  "role": "assistant"
},
    {'content': '21', 'name': 'fibonacci', 'role': 'function'}]


Response:
<OpenAIObject chat.completion id=chatcmpl-7RNt8eqTrCmvsZYclOL8OV5NOb7F2 at 0x114628bf0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 7th Fibonacci number is 21.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686761038,
  "id": "chatcmpl-7RNt8eqTrCmvsZYclOL8OV5NOb7F2",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 51,
    "total_tokens": 61
  }
}


This setup appears to only allow the model to call a single function at a time with no planning, though this could be done ala langchain with agents. (Change the question to ask two questions to see this.)

On the other hand, this model must be fine-tuned to produce function calls in that format, so perhaps it will do better at embedding them in regular output. Here's a quick test of that, though using an example that calls the same functions as basically cheating, we'll need to add some other functions to test with.

In [82]:
question = "What's the 5th Fibonacci number? What's the Collatz number of 56? What is the Collatz number of the 8th Fibonacci number?"

response = openai.ChatCompletion.create(
    model="gpt-4-0613", temperature = 0, 
    messages=[
        {"role": "system", "content": 
            """
            You may call multiple functions in a single request by returning a list of function names and parameters, in JSON format, wrapped in <eval></eval> tags.
            You can also nest function calls by setting the value of a parameter to an object with key "function_call".
            
            Example:
            User: call the function "collatz_count" with the parameter "n" set to 27, the function "fibonacci" with "n" set to 16, and the "collatz_count" of the 12th Fibonacci number, you would send the following:
            Assistant: I need to call the following functions:
                <eval>
                [
                        {
                            "name": "collatz_count",
                            "parameters": {
                                "n": 27
                            }
                        },
                        {
                            "name": "fibonacci",
                            "parameters": {
                                "n": 16
                            }
                        },
                        {
                            "name": "collatz_count",
                            "parameters": {
                                "n": {
                                    "function_call": {
                                        "name": "fibonacci",
                                        "parameters": { 
                                            "n": 12
                                        }
                                    }
                                }
                            }
                        }
                    ]
                </eval>
            """},
        {"role": "user", "content": question}],
    functions=[
        {
            "name": "collatz_count",
            "description": "Counts how many steps it takes for a number to reach 1 in the Collatz sequence.",
            "parameters": {
                "type": "object",
                "properties": {
                    "n": {
                        "type": "integer",
                        "description": "The index of the item in the Collatz sequence to get.",
                    }
                },
                "required": ["n"],
            },
        },
        {
            "name": "fibonacci",
            "description": "Gets the nth number in the Fibonacci sequence.",
            "parameters": {
                "type": "object",
                "properties": {
                    "n": {
                        "type": "integer",
                        "description": "The index of the item in the Fibonacci sequence to get.",
                    }
                },
                "required": ["n"],
            },
        }
    ],
    function_call="auto", # what happens if we disable function calling for this kind of query? Does it still pay attention to the function schemas?
)

pp.pprint(response)


<OpenAIObject chat.completion id=chatcmpl-7RO6gRVPbpyO6AMkgcJngtYzIu5iF at 0x1148f5bb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I need to call the following functions:\n<eval>\n[\n    {\n        \"name\": \"fibonacci\",\n        \"parameters\": {\n            \"n\": 5\n        }\n    },\n    {\n        \"name\": \"collatz_count\",\n        \"parameters\": {\n            \"n\": 56\n        }\n    },\n    {\n        \"name\": \"collatz_count\",\n        \"parameters\": {\n            \"n\": {\n                \"function_call\": {\n                    \"name\": \"fibonacci\",\n                    \"parameters\": { \n                        \"n\": 8\n                    }\n                }\n            }\n        }\n    }\n]\n</eval>",
        "role": "assistant"
      }
    }
  ],
  "created": 1686761878,
  "id": "chatcmpl-7RO6gRVPbpyO6AMkgcJngtYzIu5iF",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  